# Music Data Modeling Project
In this project, we apply Data Modeling with Cassandra and build an ETL pipeline using Python. We will build a Data Model around our queries that we want to get answers for. For our use case we want below answers:
1. Get details of a song that was herad on the music app history during a particular session.
2. Get songs played by a user during particular session on music app.
3. Get all users from the music app history who listened to a particular song.

# Part I. ETL Pipeline for Pre-Processing the Files

In [1]:
# import python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [3]:
# process original event csv data files
# check current working directory
print(f"current working directory: {os.getcwd()}")
# get directory of the data files
filepath = os.getcwd() + '/event_data'
# create a list of the files and collect each filepath
for root,dirs,files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    filepath_list = glob.glob(os.path.join(root,'*'))
    print(filepath_list)

current working directory: c:\Users\CaseyD\Dropbox\Baimax\PersonalDevelopment\SQL\Music_Data_Modeling
['c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\Music_Data_Modeling/event_data\\2018-11-01-events.csv', 'c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\Music_Data_Modeling/event_data\\2018-11-02-events.csv', 'c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\Music_Data_Modeling/event_data\\2018-11-03-events.csv', 'c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\Music_Data_Modeling/event_data\\2018-11-04-events.csv', 'c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\Music_Data_Modeling/event_data\\2018-11-05-events.csv', 'c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\Music_Data_Modeling/event_data\\2018-11-06-events.csv', 'c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\Music_Data_Modeling/event_data\\2018-11-07-events.csv', 'c:\\Users\\CaseyD\\Dropbox\\Baimax\\PersonalDevelopment\\SQL\\M